In [1]:
from pathlib import Path
import os
from nipype.interfaces import afni
from nipype.interfaces.utility import IdentityInterface
from nipype.pipeline.engine import Workflow, Node

In [ ]:
#interfaces namechange:
3dToutcount -->OutlierCount

#interfaces needed, line:
3dbucket, 94
align_epi_anat.py, 99
@auto_tlrc, 107
cat_matvec, 135
ABoverlap, 225
Dot, 229
1d_tool
3dTnorm, 328
gen_epi_review.py 415
gen_ss_review_scripts.py 420

#doc fix needed
OutlierCount

In [2]:
#Script is based on this command
"""afni_proc.py -subj_id $subj                                      \
        -script proc.$subj -scr_overwrite                        \
        -blocks tshift align tlrc volreg blur mask scale regress \
        -copy_anat $top_dir/FT_anat+orig                         \
        -tcat_remove_first_trs 2                                 \
        -dsets                                                   \
            $top_dir/FT_epi_r1+orig.HEAD                         \
            $top_dir/FT_epi_r2+orig.HEAD                         \
            $top_dir/FT_epi_r3+orig.HEAD                         \
        -volreg_align_to third                                   \
        -volreg_align_e2a                                        \
        -volreg_tlrc_warp                                        \
        -blur_size 4.0                                           \
        -regress_stim_times                                      \
            $top_dir/AV1_vis.txt                                 \
            $top_dir/AV2_aud.txt                                 \
        -regress_stim_labels                                     \
            vis aud                                              \
        -regress_basis 'BLOCK(20,1)'                             \
        -regress_censor_motion 0.3                               \
        -regress_opts_3dD                                        \
            -jobs 2                                              \
            -gltsym 'SYM: vis -aud' -glt_label 1 V-A             \
            -gltsym 'SYM: 0.5*vis +0.5*aud' -glt_label 2 mean.VA \
        -regress_compute_fitts                                   \
        -regress_make_ideal_sum sum_ideal.1D                     \
        -regress_est_blur_epits                                  \
        -regress_est_blur_errts                                  \
        -regress_run_clustsim no"""

['AFNItoNIFTI',
 'Allineate',
 'AutoTcorrelate',
 'Autobox',
 'Automask',
 'Axialize',
 'Bandpass',
 'BlurInMask',
 'BlurToFWHM',
 'BrickStat',
 'Calc',
 'Cat',
 'ClipLevel',
 'Copy',
 'Deconvolve',
 'DegreeCentrality',
 'Despike',
 'Detrend',
 'ECM',
 'Edge3',
 'Eval',
 'FWHMx',
 'Fim',
 'Fourier',
 'GCOR',
 'Hist',
 'Info',
 'LFCD',
 'MaskTool',
 'Maskave',
 'Means',
 'Merge',
 'Notes',
 'NwarpApply',
 'OutlierCount',
 'QualityIndex',
 'Qwarp',
 'QwarpPlusMinus',
 'ROIStats',
 'Refit',
 'Remlfit',
 'Resample',
 'Retroicor',
 'SVMTest',
 'SVMTrain',
 'Seg',
 'SkullStrip',
 'TCat',
 'TCorr1D',
 'TCorrMap',
 'TCorrelate',
 'TShift',
 'TStat',
 'To3D',
 'Unifize',
 'Volreg',
 'Warp',
 'ZCutUp',
 'Zcat',
 'Zeropad',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'base',
 'model',
 'preprocess',
 'svm',
 'utils']

In [2]:
top_dir = Path('/root/CD/AFNI_data6/FT_analysis')

In [3]:
# Define input variables
subjects = ['FT']
output_dir = top_dir/'FT.results'
working_dir = top_dir
blur_size = 4.0
stim_times =  [(top_dir/'AV1_vis.txt').as_posix(),
               (top_dir/'AV1_aud.txt').as_posix()]
stim_labels = ['vis','aud']
regress_basis = 'BLOCK(20,1)'
regress_censer_motion = 0.3
regress_opts_3dD = {"jobs":2,
                    "symlist":['vis -aud', '0.5*vis +0.5*aud'],
                    "labellist": ['V-A','mean.VA']}
tr_counts = [150,150,150]

In [4]:
# Define nodes
# lines 56-58
# lines 312
tcat = Node(afni.TCat(),
            'tcat')

outcount = Node(afni.OutlierCount(automask = True,
                                 fraction = True,
                                 polort = 3,
                                 legendre = True),
               "outcount")

# Need to make a warning node for lines 75-79

tshift = Node(afni.TShift(tzero = 0,
                          interp = "quintic"),
              "tshift")

# TODO: use nibabel or wrap 3dbucket; Done

# TODO: minimal wrapping of align_epi_anat.py Done

# TODO: minimal wrapping of @auto_tlrc; Done

# TODO: minimal wrapping of cat_matvec; Done
# lines 135 to 138
# lines 175 to 177

# TODO: Add cubic option to volreg; Done
volreg = Node(afni.Volreg(zpad = 1,
                          verbose = True),
             "volreg")
# TODO: Add overwrite flag to calc interface; Done
calc_ones = Node(afni.Calc(expr = '1'),
               "calc_ones")

# TODO: wrap cat_matvec; Done

applyxfm = Node(afni.Allineate(newgrid = 2.5),
             "allineate")

rec_xfm_cost = Node(afni.Allineate(allcostx = 'out.allcostX.txt'))

warp_follow = Node(afni.Allineate(final = 'wsinc5'))

tstat_min = Node(afni.TStat(options = "-min"),
             "tstat_min")

# TODO: Add datum option to Means interface
mean = Node(afni.Means(),
            "mean")

calc_extents = Node(afni.Calc(expr = 'step(a-0.999)'),
                    "calc_extents")

apply_extents = Node(afni.Calc(expr = 'a*b'),
                     "apply_extents")
    
blur = Node(afni.Merge(blurfwhm = int(blur_size),
                       doall = True),
            name = "blur")

dil_mask = Node(afni.Automask(dilate = 1),
                name = "dil_mask")

mask_union = Node(afni.MaskTool(union = True),
                 name = "mask_union")

# lines 218-218
# lines 234-235
resample = Node(afni.Resample(),
                name = "resample")

# lines 221-222
# lines 238-239
#mask_fill = Node(afni.MaskTool(dilate_input = '5 -5',
#                               fill_holes = True),
#                 name = "mask_fill")

# TODO: wrap 3dABoverlap; done
# lines 225-226

# TODO: wrap 3ddotl done
# lines 229-230

tstat = Node(afni.TStat(),
             name = "tstat")

scale = Node(afni.Calc(expr = "c * min(200, a/b*100)*step(a)*step(b)"),
             name = "scale")

# TODO: wrap 1d_tool.py
# lines 256-270
# lines 309
# lines 352
# lines 370-371
# lines 393-394

regress = Node(afni.Deconvolve(polort = 3,
                               num_stimts = 8,
                               jobs = 2),
               name = "regress")

tstat_mean = Node(afni.TStat(options = "-mean"),
                  "tstat_mean")

tstat_stdev = Node(afni.TStat(options = "-stdev"),
                   "tstat_stdev")

calc_tsnr = Node(afni.Calc(expr = "c*a/b"),
                 "calc_tsnr")

# TODO: wrap 3dtnorm
# lines 328

maskave = Node(afni.Maskave(quiet = True),
               "maskave")

tstat_sos = Node(afni.TStat(options = "-sos"),
                   "tstat_sos")

calc_product = Node(afni.Calc(expr = "a*b"),
                "calc_product")

tstat_sum = Node(afni.TStat(options = "-sum"),
                   "tstat_sum")

calc_dif = Node(afni.Calc(expr = "a-b"),
                "calc_dif")

# line 346
# line 347
# line 356
cat1d = Node(afni.Cat(),
             "cat1d")

fwhmx = Node(afni.FWHMx(detrend = True,
                        acf = True),
             "fwhmx")